In [23]:
# import numpy as np
# import random

# # Define TSP data (coordinates of cities)
# cities = np.array([
#     [0, 0],
#     [1, 3],
#     [2, 2],
#     [3, 1],
#     [5, 2],
#     [6, 0]
# ])
# # cities = load_dataset('dataset/large.csv')

# # Function to calculate the distance between two cities
# def distance(city1, city2):
#     return np.linalg.norm(city1 - city2)

# # Function to calculate total distance of a tour
# def tour_distance(tour):
#     total_distance = 0
#     for i in range(len(tour) - 1):
#         total_distance += distance(cities[tour[i]], cities[tour[i + 1]])
#     total_distance += distance(cities[tour[-1]], cities[tour[0]])  # Return to starting city
#     return total_distance

# # Initialize particles
# num_particles = 20
# num_iterations = 400
# particles = [random.sample(range(len(cities)), len(cities)) for _ in range(num_particles)]
# velocities = [random.sample(range(len(cities)), len(cities)) for _ in range(num_particles)]
# pBest = particles.copy()
# gBest = min(particles, key=tour_distance)

# # PSO parameters
# w = 0.5  # inertia weight
# c1 = 2.0  # cognitive weight
# c2 = 2.0  # social weight

# # PSO loop
# for iteration in range(num_iterations):
#     for i in range(num_particles):
#         # Update velocity
#         velocities[i] = w * np.array(velocities[i]) + \
#                         c1 * random.random() * (np.array(pBest[i]) - np.array(particles[i])) + \
#                         c2 * random.random() * (np.array(gBest) - np.array(particles[i]))
#         # Update position
#         particles[i] = sorted(range(len(velocities[i])), key=lambda k: velocities[i][k])
        
#         # Update personal best
#         if tour_distance(particles[i]) < tour_distance(pBest[i]):
#             pBest[i] = particles[i].copy()
            
#         # Update global best
#         if tour_distance(particles[i]) < tour_distance(gBest):
#             gBest = particles[i].copy()

# print("Best tour found:", gBest)
# print("Total distance:", tour_distance(gBest))


In [24]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
import math
import random

In [25]:
def read_cities(filepath):
    
    cities = np.loadtxt(filepath, delimiter=',')
    return cities

In [26]:
def score_solution(cities, solution):
    '''
    Calculate the total distance traveled by the given solution.
    This function scorance the salesperson would travel. Lower is better!
  es a TSP solution by computing the total
    dist  The 'solution' array must contain indices into the 'cities'
    array. Also, the 'solution' array must visit each city exactly
    once!
    '''

    if len(solution) != len(cities):
        raise Exception(('Invalid solution: len(solution) is {}, ' + \
                'but it should be {}.').format(len(solution), len(cities)))

    if set(solution) != set(range(len(cities))):
        raise Exception('Invalid solution: The solution does not ' + \
                'visit each city exactly once!')

    dist = 0.0
    for i in range(len(solution)):
        p_prev = cities[solution[i-1]]
        p_here = cities[solution[i]]
        dist += euclidean(p_prev, p_here)
    return dist

In [27]:
def create_figure():
    '''
    Creates a figure which `visualize_solution()` will draw onto.
    '''
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    return fig, axes

In [28]:
def visualize_solution(cities, solution, fig=None, axes=None, block=True):
    '''
    Visualize the solution in a 2D plot.
    The 'cities' and 'solution' arguments are the same
    as to the `score_solution()` function.
    '''
    dist = score_solution(cities, solution) if len(solution) == len(cities) else float('NaN')

    if fig is None or axes is None:
        fig, axes = create_figure()
    ax1, ax2 = axes
    fig.suptitle('Total Distance: {}'.format(dist), fontsize=20)

    ax1.clear()
    ax1.scatter(cities[:,0], cities[:,1])

    if len(solution) == len(cities):
        path = np.hstack((solution, solution[0]))  # <-- the salesperson has to return home!
    else:
        path = solution
    ax2.clear()
    ax2.plot(cities[path,0], cities[path,1])
    ax2.scatter(cities[:,0], cities[:,1])

    if block:
        while plt.fignum_exists(fig.number):
            plt.pause(0.001)
    else:
        plt.pause(0.001)

In [29]:
def random_solution(cities):
    """
    Generate a random solution for the TSP problem.
    """
    return random.sample(range(len(cities)), len(cities))

In [33]:
def pso_tsp_solver(cities, swarm_size=20, max_iter=100, w=0.5, c1=1.5, c2=1.5):
    """
    Solve the TSP problem using Particle Swarm Optimization.
    """
    num_cities = len(cities)
    best_global_solution = None
    best_global_score = float('inf')

    # Initialize particles
    particles = []
    for _ in range(swarm_size):
        particle = random_solution(cities)
        particles.append({'position': particle, 'velocity': random_solution(cities)})

    for _ in range(max_iter):
        for particle in particles:
            score = score_solution(cities, particle['position'])
            if score < best_global_score:
                best_global_score = score
                best_global_solution = particle['position']

        for particle in particles:
            # Update velocity
            for i in range(num_cities):
                r1 = random.random()
                r2 = random.random()
                particle['velocity'][i] = (w * particle['velocity'][i] +
                                           c1 * r1 * (best_global_solution[i] - particle['position'][i]) +
                                           c2 * r2 * (random.choice(best_global_solution) - particle['position'][i]))

            # Update position
            particle['position'] = sorted(range(num_cities), key=lambda x: particle['position'][x] + particle['velocity'][x])

    return best_global_solution

cities = read_cities('dataset/large.csv')

solution = pso_tsp_solver(cities)

# Calculate the total distance of the solution
total_distance = score_solution(cities, solution)

print("Final solution:", solution)
print("Total distance traveled:", total_distance)


Final solution: [758, 888, 634, 521, 668, 89, 282, 935, 848, 418, 753, 47, 663, 678, 646, 515, 906, 310, 902, 834, 590, 46, 608, 408, 923, 148, 30, 275, 435, 341, 656, 846, 313, 18, 604, 190, 392, 524, 479, 526, 542, 377, 990, 484, 68, 956, 260, 124, 51, 137, 599, 75, 868, 838, 147, 555, 189, 787, 809, 365, 785, 444, 839, 975, 946, 416, 626, 283, 686, 53, 828, 624, 442, 885, 243, 601, 760, 342, 421, 368, 560, 957, 915, 532, 28, 411, 447, 904, 49, 726, 722, 59, 26, 802, 822, 478, 674, 336, 499, 596, 942, 90, 363, 145, 613, 57, 872, 32, 409, 48, 230, 729, 426, 108, 667, 263, 531, 602, 497, 38, 629, 719, 405, 440, 423, 806, 898, 62, 544, 149, 833, 376, 157, 924, 356, 296, 36, 637, 989, 6, 932, 364, 293, 23, 495, 268, 790, 670, 591, 443, 170, 192, 407, 585, 566, 91, 469, 317, 554, 577, 878, 781, 279, 161, 949, 329, 516, 197, 619, 715, 803, 389, 208, 744, 199, 709, 214, 755, 151, 882, 684, 511, 894, 862, 471, 919, 615, 536, 496, 87, 546, 150, 234, 718, 607, 598, 97, 983, 290, 696, 581, 325,

In [31]:
486.2077578969698

486.2077578969698

In [32]:
# # Define a callback function to visualize the progress
# def visualize_path(path):
#     print("Current path:", path)

# # Run the greedy TSP solver
# solution = greedy_tsp_solver(cities, visualize_path)

# # Calculate the total distance of the solution
# total_distance = score_solution(cities, solution)

# print("Final solution:", solution)
# print("Total distance traveled:", total_distance)

Current path: [504]
Current path: [504, 222]
Current path: [504, 222, 566]
Current path: [504, 222, 566, 720]
Current path: [504, 222, 566, 720, 529]
Current path: [504, 222, 566, 720, 529, 569]
Current path: [504, 222, 566, 720, 529, 569, 732]
Current path: [504, 222, 566, 720, 529, 569, 732, 371]
Current path: [504, 222, 566, 720, 529, 569, 732, 371, 98]
Current path: [504, 222, 566, 720, 529, 569, 732, 371, 98, 395]
Current path: [504, 222, 566, 720, 529, 569, 732, 371, 98, 395, 363]
Current path: [504, 222, 566, 720, 529, 569, 732, 371, 98, 395, 363, 649]
Current path: [504, 222, 566, 720, 529, 569, 732, 371, 98, 395, 363, 649, 422]
Current path: [504, 222, 566, 720, 529, 569, 732, 371, 98, 395, 363, 649, 422, 239]
Current path: [504, 222, 566, 720, 529, 569, 732, 371, 98, 395, 363, 649, 422, 239, 969]
Current path: [504, 222, 566, 720, 529, 569, 732, 371, 98, 395, 363, 649, 422, 239, 969, 33]
Current path: [504, 222, 566, 720, 529, 569, 732, 371, 98, 395, 363, 649, 422, 239, 969, 